In [16]:
import numpy as np
import pandas as pd
import tensorflow
import keras
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [17]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0
# copies made for Neural Network later on that does not need reshaping of the data
x_test_cnn = x_test.copy()
x_train_cnn = x_train.copy()

Im going to compare a few different algorithms based on performance in classifying the Fashion_mnist dataset which contains 70000 depixelated (28x28) photos of Zalando's clothing articles separated into 10 classes.
The algorithms im going to compare are:
    <br>-Decision Tree Classifier
    <br>    -Random Forest Classifier
    <br>    -Naive Bayes
    <br>    -Support Vector Machines
    <br>    -Convolutional Neural Network
<br>I will also try to optimize those models using GridSearch

1. Decision Tree Classifier

In [18]:
# we need to reshape our data since Decision Tree Classifier works in 2D and the data comes in 3D
x_train = x_train.reshape((x_train.shape[0], -1))
x_test = x_test.reshape((x_test.shape[0], -1))
# for grid search im only gonna use the first 10% of the training data since it takes hours to compile on full set
x_train_grid_search = x_train[:6000, :]
y_train_grid_search = y_train[:6000]

model = DecisionTreeClassifier()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
cm_dtc = confusion_matrix(y_pred, y_test)
acc_dtc = accuracy_score(y_pred, y_test)
report_dtc = classification_report(y_pred, y_test)

# Grid search optimization
dtc_params = {
    'max_depth': [1, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
dtc_grid_search = GridSearchCV(DecisionTreeClassifier(), dtc_params, cv=5).fit(x_train_grid_search, y_train_grid_search)
dtc_grid_search.best_params_

{'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2}

In [19]:
model_tuned = DecisionTreeClassifier(max_depth = 10, min_samples_leaf = 1, min_samples_split = 2)
model_tuned.fit(x_train, y_train)
y_pred = model_tuned.predict(x_test)
cm_dtc_tuned = confusion_matrix(y_pred, y_test)
acc_dtc_tuned = accuracy_score(y_pred, y_test)
report_dtc_tuned = classification_report(y_pred, y_test)

In [20]:
cm_dtc

array([[733,   7,  23,  49,  15,   0, 134,   0,  12,   0],
       [  7, 954,   3,  23,   4,   1,   7,   0,   1,   0],
       [ 30,   6, 662,  20, 184,   0, 136,   0,  14,   0],
       [ 39,  17,  18, 782,  58,   2,  39,   0,   7,   0],
       [ 14,   7, 148,  60, 614,   1, 106,   0,  11,   0],
       [  1,   0,   2,   0,   1, 886,   2,  41,  12,  27],
       [160,   6, 132,  50, 119,   0, 556,   0,  20,   1],
       [  2,   0,   1,   0,   0,  58,   2, 878,   6,  57],
       [ 12,   2,   9,  15,   5,  22,  17,   5, 913,   4],
       [  2,   1,   2,   1,   0,  30,   1,  76,   4, 911]], dtype=int64)

In [21]:
cm_dtc_tuned

array([[773,  11,  12,  33,   4,   1, 135,   0,   4,   2],
       [  3, 925,   0,  10,   0,   3,   5,   0,   5,   0],
       [ 27,   4, 665,  26, 126,   0, 163,   0,  16,   1],
       [ 41,  41,   9, 814,  39,   1,  37,   0,  10,   0],
       [ 12,   7, 242,  62, 733,   1, 140,   0,  20,   0],
       [  0,   0,   0,   0,   0, 873,   4,  33,   6,  21],
       [130,   9,  63,  49,  96,   2, 499,   0,  28,   1],
       [  1,   0,   0,   0,   0,  65,   0, 924,   9,  65],
       [  9,   2,   8,   5,   2,  19,  15,   4, 900,   3],
       [  4,   1,   1,   1,   0,  35,   2,  39,   2, 907]], dtype=int64)

In [22]:
acc_dtc

0.7889

In [23]:
acc_dtc_tuned

0.8013

In [24]:
print(report_dtc)

              precision    recall  f1-score   support

           0       0.73      0.75      0.74       973
           1       0.95      0.95      0.95      1000
           2       0.66      0.63      0.65      1052
           3       0.78      0.81      0.80       962
           4       0.61      0.64      0.63       961
           5       0.89      0.91      0.90       972
           6       0.56      0.53      0.54      1044
           7       0.88      0.87      0.88      1004
           8       0.91      0.91      0.91      1004
           9       0.91      0.89      0.90      1028

    accuracy                           0.79     10000
   macro avg       0.79      0.79      0.79     10000
weighted avg       0.79      0.79      0.79     10000



In [25]:
print(report_dtc_tuned)

              precision    recall  f1-score   support

           0       0.77      0.79      0.78       975
           1       0.93      0.97      0.95       951
           2       0.67      0.65      0.66      1028
           3       0.81      0.82      0.82       992
           4       0.73      0.60      0.66      1217
           5       0.87      0.93      0.90       937
           6       0.50      0.57      0.53       877
           7       0.92      0.87      0.90      1064
           8       0.90      0.93      0.92       967
           9       0.91      0.91      0.91       992

    accuracy                           0.80     10000
   macro avg       0.80      0.80      0.80     10000
weighted avg       0.80      0.80      0.80     10000



2. Random Forest Classifier

In [27]:
model = RandomForestClassifier()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
cm_rfc = confusion_matrix(y_pred, y_test)
acc_rfc = accuracy_score(y_pred, y_test)
report_rfc = classification_report(y_pred, y_test)

# Grid search optimization
rfc_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [1, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
rfc_grid_search = GridSearchCV(RandomForestClassifier(), rfc_params, cv=5).fit(x_train_grid_search, y_train_grid_search)
rfc_grid_search.best_params_

{'max_depth': 20,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 300}

In [46]:
model_tuned = RandomForestClassifier(max_depth = 10, min_samples_leaf = 1, min_samples_split = 2, n_estimators = 300)
model_tuned.fit(x_train, y_train)
y_pred = model_tuned.predict(x_test)
cm_rfc_tuned = confusion_matrix(y_pred, y_test)
acc_rfc_tuned = accuracy_score(y_pred, y_test)
report_rfc_tuned = classification_report(y_pred, y_test)

In [28]:
cm_rfc

array([[862,   2,  13,  18,   1,   0, 154,   0,   0,   0],
       [  0, 963,   0,   2,   0,   0,   2,   0,   2,   0],
       [ 11,   1, 808,  12,  94,   0, 121,   0,   3,   0],
       [ 29,  19,  10, 905,  39,   1,  25,   0,   4,   0],
       [  3,   4, 110,  28, 812,   0,  88,   0,   5,   0],
       [  1,   0,   0,   0,   0, 958,   0,  14,   2,   8],
       [ 82,   9,  56,  33,  52,   0, 594,   0,   8,   0],
       [  0,   0,   0,   0,   0,  30,   0, 951,   4,  43],
       [ 12,   2,   3,   2,   2,   2,  16,   0, 971,   3],
       [  0,   0,   0,   0,   0,   9,   0,  35,   1, 946]], dtype=int64)

In [47]:
cm_rfc_tuned

array([[826,   4,   9,  21,   0,   0, 182,   0,   0,   0],
       [  0, 937,   0,   1,   0,   0,   1,   0,   1,   0],
       [ 20,  10, 731,  17, 102,   0, 144,   0,  13,   0],
       [ 56,  37,  10, 896,  45,   1,  40,   0,   3,   0],
       [  6,   3, 194,  37, 800,   0, 121,   0,   5,   0],
       [  1,   0,   0,   0,   0, 933,   0,  11,   2,   7],
       [ 76,   8,  50,  26,  50,   0, 491,   0,   5,   1],
       [  0,   0,   0,   0,   0,  47,   0, 920,   4,  51],
       [ 15,   1,   6,   2,   3,   3,  21,   0, 966,   2],
       [  0,   0,   0,   0,   0,  16,   0,  69,   1, 939]], dtype=int64)

In [29]:
acc_rfc

0.877

In [48]:
acc_rfc_tuned

0.8439

In [30]:
print(report_rfc)

              precision    recall  f1-score   support

           0       0.86      0.82      0.84      1050
           1       0.96      0.99      0.98       969
           2       0.81      0.77      0.79      1050
           3       0.91      0.88      0.89      1032
           4       0.81      0.77      0.79      1050
           5       0.96      0.97      0.97       983
           6       0.59      0.71      0.65       834
           7       0.95      0.93      0.94      1028
           8       0.97      0.96      0.96      1013
           9       0.95      0.95      0.95       991

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



In [49]:
print(report_rfc_tuned)

              precision    recall  f1-score   support

           0       0.83      0.79      0.81      1042
           1       0.94      1.00      0.97       940
           2       0.73      0.70      0.72      1037
           3       0.90      0.82      0.86      1088
           4       0.80      0.69      0.74      1166
           5       0.93      0.98      0.95       954
           6       0.49      0.69      0.58       707
           7       0.92      0.90      0.91      1022
           8       0.97      0.95      0.96      1019
           9       0.94      0.92      0.93      1025

    accuracy                           0.84     10000
   macro avg       0.84      0.84      0.84     10000
weighted avg       0.85      0.84      0.85     10000



3. Naive Bayes

In [31]:
model = MultinomialNB()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
cm_nb = confusion_matrix(y_pred, y_test)
acc_nb = accuracy_score(y_pred, y_test)
report_nb = classification_report(y_pred, y_test)

# Grid search optimization
nb_params = {'alpha': [0.1, 0.5, 1.0, 1.5, 2.0]}
nb_grid_search = GridSearchCV(MultinomialNB(), nb_params, cv=5).fit(x_train_grid_search, y_train_grid_search)
nb_grid_search.best_params_

{'alpha': 0.5}

In [50]:
model_tuned = MultinomialNB(alpha=0.5)
model_tuned.fit(x_train, y_train)
y_pred = model_tuned.predict(x_test)
cm_nb_tuned = confusion_matrix(y_pred, y_test)
acc_nb_tuned = accuracy_score(y_pred, y_test)
report_nb_tuned = classification_report(y_pred, y_test)

In [32]:
cm_nb

array([[776,  17,   8,  26,   0,   1, 230,   0,   7,   0],
       [  1, 872,   0,   9,   1,   0,   1,   0,   0,   0],
       [  5,  12, 569,   1, 225,   0, 138,   0,   3,   1],
       [112,  84,  13, 875, 112,   0,  85,   0,  49,   1],
       [  6,   8, 300,  25, 605,   0, 335,   0,  83,   0],
       [  0,   0,   0,   0,   0, 137,   0,  25,   3,  44],
       [ 80,   5,  85,  61,  48,   8, 161,   0,  44,  12],
       [  0,   0,   0,   0,   0, 504,   0, 920,  12, 103],
       [ 20,   2,  25,   3,   9,   6,  50,   0, 799,   1],
       [  0,   0,   0,   0,   0, 344,   0,  55,   0, 838]], dtype=int64)

In [51]:
cm_nb_tuned

array([[776,  17,   8,  27,   0,   1, 230,   0,   7,   0],
       [  1, 872,   0,   9,   1,   0,   1,   0,   0,   0],
       [  5,  12, 569,   1, 227,   0, 139,   0,   3,   1],
       [112,  84,  13, 874, 112,   0,  85,   0,  49,   1],
       [  6,   8, 300,  25, 602,   0, 332,   0,  83,   0],
       [  0,   0,   0,   0,   0, 138,   0,  25,   3,  44],
       [ 80,   5,  85,  61,  49,   8, 163,   0,  43,  12],
       [  0,   0,   0,   0,   0, 503,   0, 920,  10, 103],
       [ 20,   2,  25,   3,   9,   7,  50,   0, 802,   1],
       [  0,   0,   0,   0,   0, 343,   0,  55,   0, 838]], dtype=int64)

In [33]:
acc_nb

0.6552

In [52]:
acc_nb_tuned

0.6554

In [34]:
print(report_nb)

              precision    recall  f1-score   support

           0       0.78      0.73      0.75      1065
           1       0.87      0.99      0.93       884
           2       0.57      0.60      0.58       954
           3       0.88      0.66      0.75      1331
           4       0.60      0.44      0.51      1362
           5       0.14      0.66      0.23       209
           6       0.16      0.32      0.21       504
           7       0.92      0.60      0.72      1539
           8       0.80      0.87      0.83       915
           9       0.84      0.68      0.75      1237

    accuracy                           0.66     10000
   macro avg       0.66      0.65      0.63     10000
weighted avg       0.74      0.66      0.68     10000



In [ ]:
print(report_nb_tuned)

4. Support Vector Machine

In [35]:
model = SVC()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
cm_svm = confusion_matrix(y_pred, y_test)
acc_svm = accuracy_score(y_pred, y_test)
report_svm = classification_report(y_pred, y_test)

# Grid search optimization
svm_params = {
    'C': [0.1, 1, 2],
    'kernel': ['linear', 'poly', 'sigmoid']
}
svm_grid_search = GridSearchCV(SVC(), svm_params, cv=5).fit(x_train_grid_search, y_train_grid_search)
svm_grid_search.best_params_

{'C': 0.1, 'kernel': 'linear'}

In [55]:
model_tuned = SVC(C=0.1, kernel='linear')
model_tuned.fit(x_train, y_train)
y_pred = model_tuned.predict(x_test)
cm_svm_tuned = confusion_matrix(y_pred, y_test)
acc_svm_tuned = accuracy_score(y_pred, y_test)
report_svm_tuned = classification_report(y_pred, y_test)

In [36]:
cm_svm

array([[857,   4,  11,  27,   1,   0, 135,   0,   3,   0],
       [  0, 962,   2,   3,   1,   0,   1,   0,   1,   0],
       [ 16,   2, 816,  11,  87,   0, 104,   0,   1,   0],
       [ 28,  25,  16, 890,  32,   1,  27,   0,   5,   0],
       [  3,   3,  88,  33, 815,   0,  68,   0,   2,   0],
       [  2,   0,   0,   0,   0, 951,   0,  21,   2,  11],
       [ 85,   4,  65,  32,  61,   0, 654,   0,   4,   1],
       [  0,   0,   0,   0,   0,  33,   0, 955,   5,  37],
       [  9,   0,   2,   4,   3,   1,  11,   0, 977,   0],
       [  0,   0,   0,   0,   0,  14,   0,  24,   0, 951]], dtype=int64)

In [56]:
cm_svm_tuned

array([[828,   6,  20,  37,   0,   0, 161,   0,   6,   0],
       [  3, 959,   4,  10,   2,   0,   1,   0,   1,   0],
       [ 10,   3, 770,  16, 110,   0, 116,   0,   6,   0],
       [ 45,  25,  10, 873,  31,   1,  37,   0,   9,   0],
       [  2,   4, 108,  26, 779,   0,  92,   0,   2,   0],
       [  1,   0,   0,   0,   0, 937,   0,  29,   3,  16],
       [101,   2,  83,  35,  74,   0, 571,   0,  17,   0],
       [  0,   0,   0,   0,   0,  45,   0, 945,   4,  36],
       [ 10,   1,   5,   3,   4,   2,  22,   0, 952,   1],
       [  0,   0,   0,   0,   0,  15,   0,  26,   0, 947]], dtype=int64)

In [37]:
acc_svm

0.8828

In [57]:
acc_svm_tuned

0.8561

In [38]:
print(report_svm)

              precision    recall  f1-score   support

           0       0.86      0.83      0.84      1038
           1       0.96      0.99      0.98       970
           2       0.82      0.79      0.80      1037
           3       0.89      0.87      0.88      1024
           4       0.81      0.81      0.81      1012
           5       0.95      0.96      0.96       987
           6       0.65      0.72      0.69       906
           7       0.95      0.93      0.94      1030
           8       0.98      0.97      0.97      1007
           9       0.95      0.96      0.96       989

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



In [ ]:
print(report_svm_tuned)

5. Convolutional Neural Network

In [39]:


model = Sequential([
 Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
 Conv2D(64, (3, 3), activation='relu'),
 MaxPooling2D((2, 2)),
 Conv2D(64, (3, 3), activation='relu'),
 MaxPooling2D((2, 2)),
 Flatten(),
 Dense(64, activation='relu'),
 Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_train_cnn, y_train, epochs=10, validation_data=(x_test_cnn, y_test))
y_pred = np.argmax(model.predict(x_test_cnn), axis=-1)
cm_cnn = confusion_matrix(y_pred, y_test)
acc_cnn= accuracy_score(y_pred, y_test)
report_cnn = classification_report(y_pred, y_test)

c:\Users\Szymon\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 33s 17ms/step - accuracy: 0.7721 - loss: 0.6227 - val_accuracy: 0.8829 - val_loss: 0.3254
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 31s 17ms/step - accuracy: 0.8940 - loss: 0.2878 - val_accuracy: 0.9000 - val_loss: 0.2783
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 31s 16ms/step - accuracy: 0.9132 - loss: 0.2348 - val_accuracy: 0.9023 - val_loss: 0.2723
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 30s 16ms/step - accuracy: 0.9273 - loss: 0.1997 - val_accuracy: 0.9060 - val_loss: 0.2645
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 31s 17ms/step - accuracy: 0.9377 - loss: 0.1672 - val_accuracy: 0.9080 - val_loss: 0.2445
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 31s 16ms/step - accuracy: 0.9447 - loss: 0.1462 - val_accuracy: 0.9225 - val_loss: 0.2279
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 31s 16ms/step - accuracy: 0.9535 - loss: 0.1278 - val_accuracy: 0.9198 - val_loss: 0.2308
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 31s 16ms/step - accuracy: 0.9586 -

In [40]:
cm_cnn

array([[846,   1,  11,  13,   2,   0,  82,   0,   3,   1],
       [  0, 987,   0,   4,   0,   0,   0,   0,   1,   0],
       [ 15,   0, 875,   9,  53,   0,  43,   0,   0,   0],
       [ 10,   7,   7, 904,  22,   0,  17,   0,   5,   0],
       [  3,   0,  41,  22, 840,   0,  36,   0,   2,   0],
       [  1,   0,   0,   0,   0, 980,   0,   3,   1,   4],
       [116,   2,  64,  44,  82,   0, 815,   0,   1,   0],
       [  0,   0,   0,   0,   0,  12,   0, 956,   2,  19],
       [  9,   3,   2,   4,   1,   0,   7,   0, 984,   0],
       [  0,   0,   0,   0,   0,   8,   0,  41,   1, 976]], dtype=int64)

In [41]:
acc_cnn

0.9163

In [42]:
print(report_cnn)

              precision    recall  f1-score   support

           0       0.85      0.88      0.86       959
           1       0.99      0.99      0.99       992
           2       0.88      0.88      0.88       995
           3       0.90      0.93      0.92       972
           4       0.84      0.89      0.86       944
           5       0.98      0.99      0.99       989
           6       0.81      0.73      0.77      1124
           7       0.96      0.97      0.96       989
           8       0.98      0.97      0.98      1010
           9       0.98      0.95      0.96      1026

    accuracy                           0.92     10000
   macro avg       0.92      0.92      0.92     10000
weighted avg       0.92      0.92      0.92     10000



In [43]:
print(acc_cnn, '\n', acc_dtc, '\n', acc_nb, '\n', acc_svm, '\n', acc_rfc)

0.9163 
 0.7889 
 0.6552 
 0.8828 
 0.877
